# Determining Airline Prices
By: Chirstopher Kuzemka : [Github](https://git.generalassemb.ly)

## Problem Statement

Aviation is one of the largest industries dominating our global market today. Commercial aviation has made it possible for people to connect with each other in ways that may have been unimaginable over a century ago. However, a lot of thought must be put into the FAA standards and routes that modern planes must make today to make such connections possible.

Consider the case example where a startup airliner, known as "Kruze", wants to establish itself as a top competitor against existing airliners today. A part of this startup process focuses on understanding the costs that will come into play when managing flights. Our job as data scientists today is to help Kruze determine the minimum threshold cost the airliner must charge their passengers on a ticket class basis in order to break even with a profit. To do this, we are going to use existing flight routes (velocity and altitude data), existing data on jet fuel pricing, and existing flight ticket prices (as a prediction) to help us create a supervised learning model. 

To start, we will approach the project with the intention of expressing a minimum proof of concept. With such introduction, we will make some limitations to our study and decrease the potential for scope increase by:

- conducting an idealized thermal jet propulsion cycle for feature engineering purposes (focusing on an open Brayton cycle in particular)
- analyzing flight route data across the U.S. domestically; choosing up to 3 routes of varying sizes and suggesting their reverse flight paths as data inputs as well. 
    - **Houston, TX** to **Los Angeles, CA** (IAH - LAX)
    - **New York City, NY** to **Miami, FL** (JFK - MIA)
    - **Portland, WA** to **Chicago, IL** (PDX - ORD)
- assuming air to be treated as an ideal gas
- assuming operating engine conditions to be steady state
- assuming kinetic energy and potential energy to be negligible in our system, except at inlet and exit conditioins of jet engine itself
- assuming atmospheric temperature, pressure, and air density to be an averaged value between 0 and 15,000 meters altitude
- assuming data incorporating head or tail wind effects to be negligible
- assuming passenger weight to be negligible
- assuming external costs from the study (including food/maintenance/crew salary) to be negligible
- using price data from future flights as opposed to previous flights as previous flight pricing is not readily available


All current assumptions labeled are set to allow us to achieve (or attempt to achieve) our goal within a certain time frame, as Kruze is requiring an answer from us quickly! With this in mind, we will consider discussing how such assumptions can contribute to any error throughout our study, as well as remind ourselves that integrating negated features for future work may actually be very beneficial to us in achieveing a stronger prediction. Conducting an idealized thermal engine analysis will help us understand the average power output of a given plane's engines throughout different phases of its flight. Routes chosen throughout a variety of times and seasons will also help us determine how such elements play a role in pricing. Finally, some plane specifications (including aircraft type, number of seats it supports, as well as type/number of engines) will allow us to consider any extra technical factors for ticket pricing. 

As we are working with what is considerred to be a continuous variable, we will analyze common price trends utilizing a supervised regression model, such as Linear Regression, Logistic Regression, SVR, AdaBoosting Regression, Gradient Boosting Regression, KNNRegression, and Naive Bayes Regression. We will ultimately be using the Mean Absolute Error against our predictions to help us gauge how well our selected model predicts the price and discuss what issues may be observed from the limitations of this study.



## Executive Summary

## Table of Contents
[1.00 Data Loading](#1.00-Data-Loading)

[2.00 Superficial Analysis and History](#2.00-Superficial-Analysis-and-History)

- [2.01 Quick Check](#2.01-Quick-Check)

- [2.02 Data Documentation Exploration](#2.02-Data-Documentation-Exploration)

[3.00 Data Cleaning](#3.00-Data-Cleaning)  

[4.00 Exploratory Data Analysis and Visualization](#4.00-Exploratory-Data-Analysis-and-Visualization)

[5.00 Machine Learning Modeling and Visulalization](#5.00-Machine-Learning-Modeling-and-Visulalization)

- [3.01 Model Preparation](#3.01-Model-Preparation)

- [3.02 Modeling](#3.02-Modeling)

- [3.03 Model Selection](#3.03-Model-Selection)

- [3.04 Model Evaluation](#3.04-Model-Evaluation)

[6.00 Conclusions](#6.00-Conclusions)

[7.00 Sources and References](#7.00-Sources-and-References)

## Data Dictionary

# 1.00 Data Loading

In [80]:
import pandas as pd #imports the pandas package
import numpy as np #imports the numpy package
import matplotlib.pyplot as plt #imports the matplotlib plotting package
import seaborn as sns #imports the seaborn package

import json #imports the json package
import ast

## 1.01 Flight Tracking Data

In [2]:
current_flights = pd.read_csv('../data/current_flights.csv') #reads the current_flights csv
flight_combinations = pd.read_csv('../data/flight_combinations.csv') #reads the flight_combinations csv
flight_schedules = pd.read_csv('../data/flight_schedules.csv') #reads the flight_schedules csv

## 1.02 Pricing Data

In [147]:
monthly_pricing_2021 = pd.read_csv('../data/2021_monthly_pricing2.csv') #reads the 2021_monthly_pricing csv
june2020_to_december2020_monthlyprice = pd.read_csv('../data/june2020_to_december2020_monthlyprice.csv') #reads the 2020

## 1.03 Additional Relevant Data

In [4]:
tsa_checkpoint_travel = pd.read_excel('../data/tsa_checkpoint_travel.xlsx', sheet_name = 'Sheet1', index_col = None, usecols = 'A:C') #reads the tsa_checkpoint_travel xlsx
tsa_confirmed_cases = pd.read_excel('../data/tsa_confirmed_cases.xlsx', sheet_name = 'Sheet1', index_col = None, usecols=  'A:E') #reads the tsa_confirmed_cases xlsx

# 2.00 Data Data Cleaning and Analysis

## 2.01 Quick Check

In [5]:
def quick_check(dataframe):
    print("-------------------------------------------------------------------------------------------------")
    print(f"The head of your input dataframe is dataframe is:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.head()) #checks the head of the dataframe
    print("-------------------------------------------------------------------------------------------------")
    print(f"The tail of your input dataframe is:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.tail()) #checks the tail of the dataframe
    print("-------------------------------------------------------------------------------------------------")
    print(f"The shape of the dataframe is {dataframe.shape[0]} rows and {dataframe.shape[1]} columns.") #checks the shape of the dataframe
    print("-------------------------------------------------------------------------------------------------")
    print("The below shows whether there exist nulls in our dataframe or not:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.isnull().any()) #checks the null status of the current_flights dataframe
    print("-------------------------------------------------------------------------------------------------")
    print("The below shows the useful information to be aware of when exploring this input dataframe:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.info()) #checks the null status of the current_flights dataframe

The above function is created to conveniently conduct a quick check on the dataframe for the reader/user. Through it, we will able to see the __head__, __tail__, __shape__, __null presence__, and __important dataframe information__.

### Current Flights Data

In [6]:
quick_check(current_flights) #performs a quick check on the current_flights dataframe

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0                      faFlightID   ident prefix  type  suffix  \
0           0       DAL333-1590465975-fa-0008  DAL333    NaN  A321     NaN   
1           1  KLM601-1590468354-airline-0005  KLM601    NaN  B77W     NaN   
2           2       VIR607-1590664542-ed-0002  VIR607    NaN  B789     NaN   
3           3       DAL702-1590465982-fa-0006  DAL702    NaN  A321     NaN   
4           4  ACA572-1590468353-airline-0278  ACA572    NaN  A319     NaN   

  origin destination  timeout   timestamp  ...  lowLatitude  highLongitude  \
0   KATL        KLAX        0  1590716390  ...     32.94676      -84.44664   
1   EHAM        KLAX        0  1590711509  ...     33.95142        4.71741   
2   EGLL        KLAX        0  1590711368  ...    

#### Key takeaways fromm the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 105 rows and 26 columns. Not all columns are revealed in the head and tail of the function. 

- Our dataframe contains nulls. 

- Most of the values in our dataframe are numerical. 

### Flight Combinations Data

In [7]:
quick_check(flight_combinations) #performs a quick check on the flight combinations dataframe

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0 origin destination  0
0           0   CYHM        KJFK  1
1           1   CYUL        KORD  1
2           2   CYVR        KLAX  1
3           3   CYYZ        KIAH  2
4           4   CYYZ        KJFK  1
-------------------------------------------------------------------------------------------------
The tail of your input dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    Unnamed: 0 origin destination  0
55          55   KBUR        KPDX  1
56          56   KBWI        KPDX  1
57          57   KCVG        KPDX  1
58          58   KCVO        KPDX  2
59          59   KDEN        KPDX  3
---------------------------------------------------------------------------------

#### Key Takeaways form the above output:

- There is an `Unnamed: 0` column which does not need to be included. 

- The dataframe is 60 rows and has 4 columns. 

- The column named `0` is the final column and shows the frequency of the flight combination shown in the dataframe. 

- There are no nulls. 

- All of the values are numerical. 

### Flight Schedules Data

In [8]:
quick_check(flight_schedules)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0    ident actual_ident  departuretime  arrival_time origin  \
0           0  UAL4282      ASQ4282     1588330800    1588340820   CYUL   
1           1  ACA7591      AC27591     1588335000    1588343880   CYUL   
2           2  UAL8371      AC27591     1588335000    1588343880   CYUL   
3           3  UAL4245      ASQ4245     1588341060    1588351080   CYUL   
4           4  UAL8481      AC27595     1588353300    1588362000   CYUL   

  destination aircrafttype                                       meal_service  \
0        KORD         E75L    Business: Refreshments / Economy: Food for sale   
1        KORD         E75L  Business: Breakfast / Economy: Breakfast, Food...   
2        KORD         E75L       Business: Breakfast / Econ

#### Key takeaways fromm the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 5823 rows and 12 columns.

- Our dataframe contains nulls. 

- We have an even split of numerical and string values. 

### Monthly Pricing 2021 Data

In [148]:
quick_check(monthly_pricing_2021)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0 Quotes Places Carriers  \
0           0     []     []       []   
1           1     []     []       []   
2           2     []     []       []   
3           3     []     []       []   
4           4     []     []       []   

                                          Currencies ValidationErrors  
0  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
1  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
2  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
3  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
4  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
--------------------------------------------------------------------------------

#### Key takeaways from the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- The data is stored in json dictionaries. We will need to clean that to access more readable data.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 720 rows and 7 columns.

- Our dataframe contains nulls. 

- We have mostly string values.  

### June 2020 To December 2020 Monthly Prices Data

In [149]:
quick_check(june2020_to_december2020_monthlyprice)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0                                             Quotes  \
0           0                                                 []   
1           1  [{'QuoteId': 1, 'MinPrice': 327.0, 'Direct': F...   
2           2  [{'QuoteId': 1, 'MinPrice': 353.0, 'Direct': F...   
3           3                                                 []   
4           4                                                 []   

                                              Places  \
0                                                 []   
1  [{'PlaceId': 60987, 'IataCode': 'JFK', 'Name':...   
2  [{'PlaceId': 60987, 'IataCode': 'JFK', 'Name':...   
3                                                 []   
4                                                 []   

       

#### Key takeaways fromm the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- The data is stored in json dictionaries. We will need to clean that to access more readable data.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 5823 rows and 6 columns.

- Our dataframe contains nulls. 

- We have mostly string values. 

### TSA Checkpoint Data

In [12]:
quick_check(tsa_checkpoint_travel)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        Date  Total Traveler Throughput  Total Traveler Throughput.1
0        NaT                        NaN  (1 Year Ago - Same Weekday)
1 2020-05-26                   264843.0                      2453649
2 2020-05-25                   340769.0                      2512237
3 2020-05-24                   267451.0                      2070716
4 2020-05-23                   253190.0                      2124825
-------------------------------------------------------------------------------------------------
The tail of your input dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
         Date  Total Traveler Throughput Total Traveler Throughput.1
83 2020-03-05                  21300

#### Key takeaways fromm the above output:

- There is a `Date` column not currently set as the index of the dataframe. We will consider making the `Date` column the index. 

- The first row seems to contain header information. We will append the appropriate values into the header. 

- Our dataframe is 88 rows and 3 columns.

- Our dataframe contains nulls. 

- We have an even split of string, float, and datetime values. 

### TSA Confirmed Cases Data

In [13]:
quick_check(tsa_confirmed_cases)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                          Airport  Total Confirmed Cases  \
0               ACY - Atlantic City International                      1   
1                        AIA - Alliance Municipal                      1   
2                      ALW - Walla Walla Regional                      1   
3       AMA - Rick Husband Amarillo International                      1   
4  ATL - Hartsfield-Jackson Atlanta International                     17   

   TSA Screening Officers  Non-Screening Employees  \
0                     1.0                      NaN   
1                     1.0                      NaN   
2                     1.0                      NaN   
3                     1.0                      NaN   
4                    16.0 

#### Key takeaways fromm the above output:

- The data columns are separated above with a `\`.

- Our dataframe is 78 rows and 5 columns.

- Our dataframe contains nulls. 

- We have string, float, and datetime values. 

## 2.02 Data Documentation and Exploration

### What is the "Flight Data"?

The "Flight Data" is referring to the the dataframes showcasing the current flights (`current_flights`), flight combinations (`flight_combinations`), and flight schedules(`flight_schedules`). They each preside in the order which they were constructed, where the `current_flights` dataframe was collected first, then the `flight_combinations`, then the `flight_schedules`. All of the flight data collected stemmed from the root dataframe for this study: `current_flights`. Below are the data dictionaries for each dataframe after their cleaning process showcased in [3.00 Data Cleaning](#3.00-Data-Cleaning). The data dictionaries are to help us understand what each dataframe's value represents as the airline language can be non-intuitive to understand. Some of the values are also representative of API utilized to gather the data. 

### How was the Flight Data Gathered?

### What Biases Play a Role in the Data? What Compromises were Made?

### What is the Price Data?

### How was the Price Data Gathered?

### What Biases Play a Role in the Data? What Compromises were Made?

### What are the Other Relevant Data?

### How was the Other Relevant Data Gathered?

### What Biases Play a Role in the Data? What Compromises were Made?

# 3.00 Data Cleaning

## 3.01 Flight Data

### Datatype Conversion

### Null Analysis

In [207]:
def null_analysis(dataframe):
    print("-------------------------------------------------------------------------------------------------")
    print("The below shows whether there exist nulls in our dataframe or not:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.isnull().any()) #checks the null status of the current_flights dataframe
    print("-------------------------------------------------------------------------------------------------")
    print("The below shows the mean of nulls existing in a dataframe:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe.isnull().mean().sort_values(ascending = False))
    print("-------------------------------------------------------------------------------------------------")
    print(f"The column with the most nulls is the '{dataframe.isnull().mean().sort_values(ascending = False).index[0]}' column with a null percentage of {dataframe.isnull().mean().sort_values(ascending = False).iloc[0]*100}%.")
    print("-------------------------------------------------------------------------------------------------")
    highest_null_mask = dataframe[str(dataframe.isnull().mean().sort_values(ascending = False).index[0])].isnull() == True
    print("The below shows the most masked null dataframe for more clear understanding:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe[highest_null_mask])
    print("-------------------------------------------------------------------------------------------------")
    anti_highest_null_mask = dataframe[str(dataframe.isnull().mean().sort_values(ascending = False).index[0])].isnull() == False
    print("The below shows the most masked null dataframe, where nulls are filtered out, for more clear understanding:")
    print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print(dataframe[anti_highest_null_mask])

### One-Hot Encoding

## 3.02 Price Data

A plethora of price data was collected using the Skyscanner API. However, due to complications, some prices were found to be null for certain dates. With such limitations, lots of other data were collected to help us fill in the gaps for the missing values. We will explore both dataframes for pricing to help us analyze appropriate prices per flight. To start our cleaning process, we will re-state our `quick_check` function and re-state our key takeaways from earlier in the study. 

In [150]:
quick_check(monthly_pricing_2021)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0 Quotes Places Carriers  \
0           0     []     []       []   
1           1     []     []       []   
2           2     []     []       []   
3           3     []     []       []   
4           4     []     []       []   

                                          Currencies ValidationErrors  
0  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
1  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
2  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
3  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
4  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...              NaN  
--------------------------------------------------------------------------------

#### Key takeaways from the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- The data is stored in json dictionaries. We will need to clean that to access more readable data.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 720 rows and 7 columns.

- Our dataframe contains nulls. 

- We have mostly string values.  

In [152]:
quick_check(june2020_to_december2020_monthlyprice)

-------------------------------------------------------------------------------------------------
The head of your input dataframe is dataframe is:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
   Unnamed: 0                                             Quotes  \
0           0                                                 []   
1           1  [{'QuoteId': 1, 'MinPrice': 327.0, 'Direct': F...   
2           2  [{'QuoteId': 1, 'MinPrice': 353.0, 'Direct': F...   
3           3                                                 []   
4           4                                                 []   

                                              Places  \
0                                                 []   
1  [{'PlaceId': 60987, 'IataCode': 'JFK', 'Name':...   
2  [{'PlaceId': 60987, 'IataCode': 'JFK', 'Name':...   
3                                                 []   
4                                                 []   

       

#### Key takeaways fromm the above output:

- The dataframe is large and denotes separations with a `\` symbol.

- The data is stored in json dictionaries. We will need to clean that to access more readable data.

- There is an `Unnamed: 0` column in our dataframe which is not necessary to include. We will remove this in our cleaning.

- Our dataframe is 5823 rows and 6 columns.

- Our dataframe contains nulls. 

- We have mostly string values. 

### Unpacking the Monthly Pricing Dataframe and Null Analysis

Due to the Skyscanner's method of data scraping, our data is nested inside of dictionaries. We are going to need to clean up the dictionaries and most likely export a lot more hidden data from these dictionaries. We are prepared to create more dataframes out of these dictionaries. To make dictionary unpacking process more efficient, we are also going to have to perform a null analysis on all of the data so help us recognize any `NaNs` in our dataframe. Once such `NaNs` are recognized, we can determine what would be the most appropriate action to substitute a value for such `NaNs` OR completely eliminate them entirely based on their importance. 

First we are going to drop the `Unnamed: 0` column through the above defined function to ease our dataframe cleaning process across multiple dataframes. 

In [153]:
def drop_unnamed(dataframe):
    dataframe.drop(columns = 'Unnamed: 0', inplace = True)

In [156]:
drop_unnamed(monthly_pricing_2021)
drop_unnamed(june2020_to_december2020_monthly_price)

Let's do some null analysis. Our Quotes are our most important columns. So We will analyze how many nulls there are. 

In [208]:
null_analysis(monthly_pricing_2021)

-------------------------------------------------------------------------------------------------
The below shows whether there exist nulls in our dataframe or not:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Quotes              True
Places              True
Carriers            True
Currencies          True
ValidationErrors    True
dtype: bool
-------------------------------------------------------------------------------------------------
The below shows the mean of nulls existing in a dataframe:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ValidationErrors    0.966667
Currencies          0.033333
Carriers            0.033333
Places              0.033333
Quotes              0.033333
dtype: float64
-------------------------------------------------------------------------------------------------
The column with the most nulls is the 'ValidationErrors' column with a null percentag

From the above, our `ValidationErrors` column contains the most nulls in our dataframe. With respect to the context of our study, keeping these `ValidationErrors` column is unnecessary as it is only an indicator for a successful query search. We will drop it from the dataframe. Before doing so, it would best to also interpret the pecentage of `ValidationErrors` nulls and comapre against the percentage nulls of the rest of the columns in our dataframe. The rest of our dataframe is presenting approximately 3.3% nulls. Through the final print statement from the above output of the dataframe, we notice that the dataframe showing values for true `ValidationErrors` is expressing nulls across the entire rows of such data. We can confidently state that the data rest of the dataframe showing nulls are missing not at random ([MNAR](https://www.theanalysisfactor.com/missing-data-mechanism/)) in this instance. We cannot identify what is triggering the validation errors in the first place through a large data scrape, but we do know the validation errors to be present for every row of `NaNs`; we know the two are related and can confidently state that for every true instance of a validation error, we can expect a row to be empty. 

In summary, we know that there is no need to keep the `ValidationErrors` in our study. With entire rows missing form the dataframe, we will not be able to make any meaningful identification for the missing data or find a way to impute such values. Therefore, we will drop such rows. The cost of damage for this study is minimal with only a loss of approximately 3.3% of our values. Any further errors recognized will consider this dataframe drop as a culprit, during the mass dataframe concatenation conducted in a later section (partciularly combatting an instance where a flight combination may not be read for the study).

To efficiently do this, we will drop nulls across the most improtant target feature column which will shave approximately 3.3 percent of our dataframe. We will then drop the `ValidationErrors` column.   

In [210]:
monthly_pricing_2021.dropna(subset = ['Quotes'], inplace = True) 

In [212]:
monthly_pricing_2021.drop(columns = 'ValidationErrors', inplace = True)

In [213]:
null_analysis(monthly_pricing_2021)

-------------------------------------------------------------------------------------------------
The below shows whether there exist nulls in our dataframe or not:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Quotes        False
Places        False
Carriers      False
Currencies    False
dtype: bool
-------------------------------------------------------------------------------------------------
The below shows the mean of nulls existing in a dataframe:
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Currencies    0.0
Carriers      0.0
Places        0.0
Quotes        0.0
dtype: float64
-------------------------------------------------------------------------------------------------
The column with the most nulls is the 'Currencies' column with a null percentage of 0.0%.
-------------------------------------------------------------------------------------------------
The below shows

Conducting a second null_analysis on the dataframe shows that nulls no longer exist in a manner that is recognized by python. However, this does not mean we aren't missing any data. Through some of the above output, we can intuitively recognize that we are indeed missing some quotes through successful query searches. Let's analyze why python is misinterpreting such nulls. To do this, we will analyze an "empty" element in `monthly_pricing_2021[Quotes]` to see what python is recognizing, as well as a "filled" element.  

In [216]:
print(type(monthly_pricing_2021.loc[0, 'Quotes']))
print('- - - - - - - - - - - - - - - - - - - -')
print(type(monthly_pricing_2021.loc[715, 'Quotes']))

<class 'str'>
- - - - - - - - - - - - - - - - - - - -
<class 'str'>


From the above outputs, we can see that each element is being recognized as a string. This means the perceived "empty" elements are expressing a True Boolean state output. We will convert such strings to their perceived appropriate outputs with the below function.  

In [227]:
monthly_pricing_2021['Quotes'] = monthly_pricing_2021['Quotes'].apply(lambda element: ast.literal_eval(element))

Let's inspect a random element...

In [236]:
monthly_pricing_2021 = monthly_pricing_2021.apply(lambda element: ast.literal_eval(element))

ValueError: malformed node or string: 0                                                     []
1                                                     []
2                                                     []
3                                                     []
4                                                     []
                             ...                        
715    [{'QuoteId': 1, 'MinPrice': 79.0, 'Direct': Tr...
716                                                   []
717    [{'QuoteId': 1, 'MinPrice': 120.0, 'Direct': F...
718                                                   []
719                                                   []
Name: Quotes, Length: 696, dtype: object

In [235]:
monthly_pricing_2021.loc[715, 'Quotes'][0]

{'QuoteId': 1,
 'MinPrice': 79.0,
 'Direct': True,
 'OutboundLeg': {'CarrierIds': [1065],
  'OriginId': 47118,
  'DestinationId': 74728,
  'DepartureDate': '2021-02-01T00:00:00'},
 'QuoteDateTime': '2020-06-01T13:48:00'}

We must remind ourselves that this is a part of our features co

In [115]:
string = monthly_pricing_2021.loc[715, 'Quotes']
result = ast.literal_eval(string)

string2 = monthly_pricing_2021.loc[717, 'Quotes']
result2 = ast.literal_eval(string2)



test_df = pd.DataFrame(result[0], columns = result[0].keys())
test2_df = pd.DataFrame(result2[0], columns = result2[0].keys())

togetherframe = pd.concat([test_df, test2_df])

In [46]:
monthly_pricing_2021.loc[715, 'Quotes']

"[{'QuoteId': 1, 'MinPrice': 79.0, 'Direct': True, 'OutboundLeg': {'CarrierIds': [1065], 'OriginId': 47118, 'DestinationId': 74728, 'DepartureDate': '2021-02-01T00:00:00'}, 'QuoteDateTime': '2020-05-31T11:23:00'}]"

In [34]:
monthly_pricing_2021.loc[716, 'Quotes']

'[]'

#### Test on Monthly

### Null Analysis

### Datatype Conversion



### One-Hot Encoding

## 3.03 Other Relevant Data

## 3.04 Creating the Model Dataframe

## Conclusions and Future Work

For the future, consider incorporating weather data, randomized passenger weight data, incorporate the dynamic changes in fuel/mass ratio throughout a flight, incorporate some demographical passenger data, more routes, the ability for the problem to become a UI tool rather than just a study.